In [2]:
import pandas as pd

In [3]:
harsh_braking_data = pd.read_csv("Total_harsh_braking.csv")

In [4]:
harsh_braking_data.tail()

,Bus no.,Event Time,Latitude,Longitude,count,count_unique_bus,count_current_bus
197375,27,31.07.2018 23:41:24,23.064428,72.642440,10,5,4
197376,27,31.07.2018 23:41:28,23.064428,72.642440,10,5,4
197377,27,31.07.2018 23:44:46,23.064428,72.642440,10,5,4
197378,205,31.07.2018 23:44:57,23.068625,72.582002,33,11,1
197379,24,31.07.2018 23:57:25,23.129200,72.584464,63,12,3


In [5]:
ahmedabad_data = pd.read_csv("Events Report of Drivers in Ahmedabad_16-07 to 31-07.csv")
ahmedabad_driver_data_harsh = ahmedabad_data[ahmedabad_data["Event Name"] == "Harsh Braking"]
ahmedabad_driver_data_harsh.reset_index(drop=True, inplace=True)

In [6]:
ahmedabad_driver_data_harsh.tail()

,Event Name,Event Time,Event Value (Upper limit =51km/h),Event Unit,Bus no.,Route Name,Stop Code,Stop Name,Latitude,Longitude
197375,Harsh Braking,31.07.2018 23:41:24,0.451,m/s2,27,8U_Iskcon - Naroda Gam_NORMAL,313,Naroda S. T. Workshop,23.064428,72.642440
197376,Harsh Braking,31.07.2018 23:41:28,0.298,m/s2,27,8U_Iskcon - Naroda Gam_NORMAL,313,Naroda S. T. Workshop,23.064428,72.642440
197377,Harsh Braking,31.07.2018 23:44:46,0.322,m/s2,27,8U_Iskcon - Naroda Gam_NORMAL,313,Naroda S. T. Workshop,23.064428,72.642440
197378,Harsh Braking,31.07.2018 23:44:57,0.442,m/s2,205,101_R.T.O(Circular)_NORMAL,4,Ranip Cross Road,23.068625,72.582002
197379,Harsh Braking,31.07.2018 23:57:25,0.548,m/s2,24,7U_Zundal Circle - Narol_NORMAL,253,DCIS Circle,23.129200,72.584464


In [7]:
harsh_braking_data = harsh_braking_data.merge(
    ahmedabad_driver_data_harsh[["Event Time", "Bus no.", "Latitude", "Longitude", "Route Name"]],
    on=["Bus no.", "Event Time", "Latitude", "Longitude"])

In [8]:
harsh_braking_data[["count", "count_unique_bus", "count_current_bus"]].describe()

,count,count_unique_bus,count_current_bus
count,191092.000000,191092.000000,191092.000000
mean,128.651848,8.999451,81.984002
std,202.099982,7.411995,193.830726
min,1.000000,0.000000,1.000000
25%,31.000000,3.000000,5.000000
50%,77.000000,8.000000,16.000000
75%,135.000000,14.000000,72.000000
max,1559.000000,31.000000,1485.000000


In [9]:
count = {}
count_upper = {}
def cal(dataset):
    global count, count_upper
    for i in range(10, 101, 10):
        val = (dataset[6]*100)/dataset[4]
        if val < i:
            count[i] = count.get(i, 0) + 1
        else:
            count_upper[i] = count_upper.get(i, 0) + 1
harsh_braking_data.apply(cal, axis=1)
count, count_upper

({10: 40580,
  20: 63894,
  30: 81324,
  40: 93870,
  50: 101640,
  60: 110958,
  70: 118188,
  80: 125686,
  90: 138595,
  100: 170652},
 {10: 150512,
  20: 127198,
  30: 109768,
  40: 97222,
  50: 89452,
  60: 80134,
  70: 72904,
  80: 65406,
  90: 52497,
  100: 20440})

In [10]:
harsh_braking_data[harsh_braking_data["count"] <= 10][["count", "count_unique_bus", "count_current_bus"]].count()

count                18685
count_unique_bus     18685
count_current_bus    18685
dtype: int64

In [11]:
danger = {"lat": [], "long": [], "route":[]}
def danger_cal(dataset):
    global danger
    if dataset[4] > 10 and dataset[5] > 1 and ((dataset[6]*100)/dataset[4] < (100 - dataset[5]*10)):
        danger["lat"].append(dataset[2])
        danger["long"].append(dataset[3])
        danger["route"].append(dataset[7])        
        
harsh_braking_data.apply(danger_cal, axis=1)
print("done")

done


In [12]:
danger_zone = pd.DataFrame(danger)

In [97]:
# 30mt danger zone
danger_zone.to_csv("danzer_zone_30_mt.csv", index=False)

## Plot zones on map

In [13]:
data = danger_zone[danger_zone["route"] == "9S_Bhavsar Hostel - Gota Cross Road_NORMAL"].reset_index(drop=True)

In [14]:
data.tail()

,lat,long,route
62,23.062253,72.553069,9S_Bhavsar Hostel - Gota Cross Road_NORMAL
63,23.062867,72.532757,9S_Bhavsar Hostel - Gota Cross Road_NORMAL
64,23.082037,72.526696,9S_Bhavsar Hostel - Gota Cross Road_NORMAL
65,23.062228,72.553039,9S_Bhavsar Hostel - Gota Cross Road_NORMAL
66,23.057647,72.539115,9S_Bhavsar Hostel - Gota Cross Road_NORMAL


In [15]:
from math import radians, cos, sin, asin, sqrt

def haversine(long1, lat1, long2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])

    # haversine formula 
    dlong = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlong/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r

In [36]:
import gmplot
gmap = gmplot.GoogleMapPlotter(data["lat"][0], data["long"][0], 16)


# gmap.plot(data["lat"], data["long"], 'cornflowerblue', edge_width=8)

gmap.heatmap(data["lat"], data["long"], opacity=1, radius=15)
for i in range(len(data)):
    radius = 0.500 # in kilometer
    a = 0
    if i != len(data)-1:
        for j in range(len(data)):
            a = haversine(data["long"][i], data["lat"][i], data["long"][j], data["lat"][j])
            if a <= radius:
                continue
    gmap.marker(data["lat"][i], data["long"][i], 'red', title="Danger Zone")
    gmap.circle(data["lat"][i], data["long"][i], radius=15, color="red", opacity=0)

gmap.draw("mymap.html")